In [2]:
import os
from dotenv import load_dotenv
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


# Load environment variables from a .env file
load_dotenv("/home/azureuser/azure-ai-agent-workshop/.env")

True

In [3]:
model_client=AzureOpenAIChatCompletionClient(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZUER_OPENAI_API_VERSION")
        )

In [4]:
# define Tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

# define Agent
weather_agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    description="一个通过使用工具为用户提供天气信息的智能体。",
    system_message="你是一个乐于助人的AI智能助手。擅长使用工具解决任务。任务完成后，回复<stop autogenworkshop>",
    tools=[get_weather],
)


In [5]:
# Define a termination condition. End the conversation if a specific text is mentioned.
termination = TextMentionTermination("<stop autogenworkshop>")

# Define Team with type set to RoundRobinGroupChat
agent_team = RoundRobinGroupChat(participants=[weather_agent], termination_condition=termination, max_turns=3)


In [8]:
 # 1、run() method to run the team
result = await agent_team.run(task="how about the weather of shanghai?")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='how about the weather of shanghai?', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=282, completion_tokens=14), content=[FunctionCall(id='call_z8plz9kh1ZZmVIlrgohCXiHK', arguments='{"city":"Shanghai"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in Shanghai is 73 degrees and Sunny.', call_id='call_z8plz9kh1ZZmVIlrgohCXiHK')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='weather_agent', models_usage=None, content='The weather in Shanghai is 73 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=318, completion_tokens=13), content='The weather in Shanghai is currently 73 degrees and sunny.', type='TextMessage'), TextMessage(source='wea

In [6]:
# 2、run_stream()metho to run team
async for message in agent_team.run_stream(task="how about the weather of shanghai?"):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='how about the weather of shanghai?' type='TextMessage'
source='weather_agent' models_usage=RequestUsage(prompt_tokens=83, completion_tokens=14) content=[FunctionCall(id='call_kErrHwcmFDsHVN3dogVTth5n', arguments='{"city":"Shanghai"}', name='get_weather')] type='ToolCallRequestEvent'
source='weather_agent' models_usage=None content=[FunctionExecutionResult(content='The weather in Shanghai is 73 degrees and Sunny.', call_id='call_kErrHwcmFDsHVN3dogVTth5n')] type='ToolCallExecutionEvent'
source='weather_agent' models_usage=None content='The weather in Shanghai is 73 degrees and Sunny.' type='ToolCallSummaryMessage'
source='weather_agent' models_usage=RequestUsage(prompt_tokens=119, completion_tokens=13) content='The weather in Shanghai is currently 73 degrees and sunny.' type='TextMessage'
source='weather_agent' models_usage=RequestUsage(prompt_tokens=138, completion_tokens=27) content='天气情况是上海的阳光明媚，气温为73华氏度。\n\n<stop autogenworkshop>' type='TextMe

In [7]:
# Method runs the team and uses the official Console tool to output in the proper format
# 3、run_stream()
stream = agent_team.run_stream(task="上海的天气如何?")
await Console(stream)

---------- user ----------
上海的天气如何?
---------- weather_agent ----------
[FunctionCall(id='call_MpihyYGDRToFj1nioIXMNtYS', arguments='{"city":"上海"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in 上海 is 73 degrees and Sunny.', call_id='call_MpihyYGDRToFj1nioIXMNtYS')]
---------- weather_agent ----------
The weather in 上海 is 73 degrees and Sunny.
---------- weather_agent ----------
上海的天气现在是73华氏度，并且天气晴朗。
---------- weather_agent ----------
上海的天气目前为73华氏度，阳光明媚。<stop autogenworkshop>


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='上海的天气如何?', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=181, completion_tokens=14), content=[FunctionCall(id='call_MpihyYGDRToFj1nioIXMNtYS', arguments='{"city":"上海"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in 上海 is 73 degrees and Sunny.', call_id='call_MpihyYGDRToFj1nioIXMNtYS')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='weather_agent', models_usage=None, content='The weather in 上海 is 73 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=217, completion_tokens=16), content='上海的天气现在是73华氏度，并且天气晴朗。', type='TextMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=239, completion_tokens=24), 